## Минимальный необходимый размер выборки

In [3]:
var_control = 2000
var_test = 2000
za_2 = -1.96
zb = -0.842
mde = 60

#минимальный размер выборки
n = ((var_control**2 + var_test**2) * (za_2 + zb)**2)/mde**2
n = round(n)
print(f'n: {n}')

n: 17447


In [2]:
orders = 5+0+2+3+4+2+3+1+0+0
orders

In [3]:
revenue = 15000	+ 0	+ 4000	+ 4000	+ 7000	+ 1500	+ 2500	+ 1000	+ 0	+ 0
revenue

35000

In [4]:
arpu = revenue / 10
arpu

3500.0

In [5]:
arppu = revenue / 7
arppu

5000.0

In [7]:
# Average Order Value
aov = revenue / orders
aov

1750.0

In [8]:
(21476000 + 19748920 + 20569785 + 21444444) / 4

20809787.25

In [12]:
var_historical = 2000
za_2 = -1.96
zb = -0.842
mde = 60
n = ((2*var_historical) * (za_2 + zb)**2)/mde**2
n

8.723559999999999

In [14]:
revenue = 400+800+200+600+350+1000+900+1250+300+200
revenue
arppu = revenue / 6

In [15]:
arppu

1000.0

In [17]:
aov = revenue / 10
aov

600.0

In [18]:
arppu = 300
arpu = 3

arpu / arppu

0.01

In [23]:
(2875+2491)/2

2683.0